In [1]:
import os
import glob
import hydromt
from hydromt import DataCatalog
import xarray as xr
import numpy as np
from os.path import join
import geopandas as gpd
import pandas as pd
from hydromt_sfincs import SfincsModel, utils

In [2]:
''' 
Script:
Author: L Grimley
Last Updated: 8/20/24

Description: 


Inputs:

Outputs:
For all runs, the ensemble mean and the ensemble max

- pgw_drivers_classified.nc
- pgw_compound_extent.nc
- pgw_drivers_classified_cellCount.csv
- pgw_WL_maxCmpd_minus_maxIndiv.nc

'''

' \nScript:\nAuthor: L Grimley\nLast Updated: 8/20/24\n\nDescription: \n\n\nInputs:\n\nOutputs:\nFor all runs, the ensemble mean and the ensemble max\n\n- pgw_drivers_classified.nc\n- pgw_compound_extent.nc\n- pgw_drivers_classified_cellCount.csv\n- pgw_WL_maxCmpd_minus_maxIndiv.nc\n\n'

In [3]:
# Function Outputs:
#       da_classified         a data array with the zsmax attributed to processes (codes 0 to 4)
#       fld_cells_by_driver   a numpy array with the total number of grid cells for each process
#       da_compound           a data array of the cells where compound flooding occurred (e.g., 0 and 1)
#       da_diff               a data array of the diff in waterlevel compound minus max. single driver

def classify_zsmax_by_driver(da, compound_key, runoff_key, coastal_key, name_out, hmin, mask):
    # Calculate the max water level at each cell across the coastal and runoff drivers
    da_single_max = da.sel(run=[runoff_key, coastal_key]).max('run')
    
    # Calculate the difference between the max water level of the compound and the max of the individual drivers
    da_diff = (da.sel(run=compound_key) - da_single_max).compute()
    da_diff.name = 'diff in waterlevel compound minus max. single driver'
    da_diff.attrs.update(unit='m')

    # Create masks based on the driver that caused the max water level given a depth threshold hmin
    compound_mask = da_diff > hmin
    coastal_mask = da.sel(run=coastal_key).fillna(0) > da.sel(run=[runoff_key]).fillna(0).max('run')
    runoff_mask = da.sel(run=runoff_key).fillna(0) > da.sel(run=[coastal_key]).fillna(0).max('run')
    assert ~np.logical_and(runoff_mask, coastal_mask).any()
    da_classified = (xr.where(coastal_mask, x=compound_mask + 1, y=0)
                     + xr.where(runoff_mask, x=compound_mask + 3, y=0)).compute()
    da_classified.name = name_out
    
    # Calculate the number of cells that are attributed to the different drivers
    unique_codes, fld_area_by_driver = np.unique(da_classified.data, return_counts=True)

    # Mask out the water bodies!!!
    da_classified_masked = da_classified.where(mask == 0.0)
    unique_codes, fld_area_by_driver_Mask = np.unique(da_classified_masked.data, return_counts=True)
    
    # Return compound only locations
    da_compound = xr.where(compound_mask, x=1, y=0)
    da_compound.name = name_out

    return da_classified, fld_area_by_driver, da_compound, da_diff, fld_area_by_driver_Mask

In [4]:
# Load a SFINCS model and get elevation
yml_base = r'Z:\Data-Expansion\users\lelise\data\data_catalog_BASE_Carolinas.yml'
root = r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter2_PGW\sfincs\01_AGU2023\future_florence\future_florence_ensmean'
mod = SfincsModel(root=root, mode='r', data_libs=[yml_base])
dep = mod.grid['dep']

# Load coastal waterbody shapefile
coastal_wb = mod.data_catalog.get_geodataframe('carolinas_coastal_wb')
coastal_wb = coastal_wb.to_crs(mod.crs)
coastal_wb_clip = coastal_wb.clip(mod.region)
coastal_wb_clip['mask'] = 1.0
mask1 = dep.raster.rasterize(coastal_wb_clip, "mask", nodata=0.0, all_touched=False)
carolinas_nhd_area_rivers = mod.data_catalog.get_geodataframe('carolinas_nhd_area_rivers', geom=mod.region)
carolinas_nhd_area_rivers = carolinas_nhd_area_rivers.to_crs(mod.crs)
carolinas_nhd_area_rivers['mask'] = 1.0
mask2 = dep.raster.rasterize(carolinas_nhd_area_rivers, "mask", nodata=0.0, all_touched=True)
mask = (mask1 + mask2).compute()
mask = xr.where(cond = mask > 0.0, x= 1.0,y=0.0)

In [5]:
work_dir = r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter2_PGW\sfincs\03_OBS\analysis_2'
out_dir = os.path.join(work_dir, 'driver_analysis')
if os.path.exists(out_dir) is False:
    os.makedirs(out_dir)
os.chdir(out_dir)

storms = ['flor', 'floy', 'matt']
climates = ['pres', 'fut']
hmin = 0.05  # minimum difference between the individual and compound drivers

In [6]:
''' Parse the drivers for each simulation '''
# Load the peak water levels for all simulations
zsmax_file = os.path.join(work_dir, 'zsmax', 'pgw_zsmax.nc')
da_zsmax = xr.open_dataarray(zsmax_file)

In [7]:
fld_cells = pd.DataFrame()  # dataframe populated with total flooded area
fld_cells_mask = pd.DataFrame()  # dataframe populated with total flooded area
fld_da_compound = []  # populated with data arrays of the compound areas for each run
fld_da_classified = []
fld_da_diff = []
cc_run_ids = []
run_ids = []  # keep track of the run IDs and their order
# Loop through and classify flooding by driver
for storm in storms:
    nruns = 8
    if storm == 'matt':
        nruns = 7
    for climate in climates:
        if climate == 'pres':
            # Get the model names to query zsmax results from data array
            compound_key, runoff_key, coastal_key = [f'{storm}_{climate}_compound',
                                                     f'{storm}_{climate}_runoff',
                                                     f'{storm}_{climate}_coastal']
            nameout = f'{storm}_{climate}'

            # Attribute max water levels to coastal, runoff or compound processes
            out = classify_zsmax_by_driver(da=da_zsmax,  # data array with zsmax for all simulations
                                           compound_key=compound_key,
                                           runoff_key=runoff_key,
                                           coastal_key=coastal_key,
                                           name_out=nameout,
                                           hmin=hmin,  # depth difference threshold for compound vs. individual
                                           mask=mask
                                           )
            da_classified, fld_cells_by_driver, da_compound, da_diff, fld_cells_by_driver_Mask = out

            # Append all the output from the function above to lists
            fld_da_classified.append(da_classified)
            fld_cells[f'{da_compound.name}'] = fld_cells_by_driver
            fld_cells_mask[f'{da_compound.name}'] = fld_cells_by_driver_Mask
            fld_da_compound.append(da_compound)
            fld_da_diff.append(da_diff)
            run_ids.append(f'{da_compound.name}')
            print(da_compound.name)

        if climate == 'fut':
            for sf in np.arange(1, nruns, 1):
                for slr in np.arange(1, 6, 1):
                    compound_key, runoff_key, coastal_key = [f'{storm}_{climate}_SF{sf}_SLR{slr}_compound',
                                                             f'{storm}_{climate}_SF{sf}_runoff',
                                                             f'{storm}_{climate}_SF{sf}_SLR{slr}_coastal']

                    nameout = f'{storm}_{climate}_SF{sf}_SLR{slr}'
                    out = classify_zsmax_by_driver(da=da_zsmax,
                                                   compound_key=compound_key,
                                                   runoff_key=runoff_key,
                                                   coastal_key=coastal_key,
                                                   name_out=nameout,
                                                   hmin=hmin,
                                                   mask=mask
                                                   )
                    # Append the classified data array (e.g., coastal, runoff, compound) to the larger list
                    fld_da_classified.append(da_classified)
                    # Append the count of flooded cells by driver to the dataframe
                    fld_cells[f'{da_compound.name}'] = fld_cells_by_driver
                    # Append the count of flooded cells by driver using a mask to the dataframe
                    fld_cells_mask[f'{da_compound.name}'] = fld_cells_by_driver_Mask
                    # Append the compound extent data array to the larger list
                    fld_da_compound.append(da_compound)
                    # Append the difference between the compound and the max single driver data array to the larger list
                    fld_da_diff.append(da_diff)
                    # Append the run ID to the larger list, used for saving as a run index
                    run_ids.append(f'{da_compound.name}')
                    # Write the compound extent to a tif for viewing
                    da_compound.raster.to_raster(f'{da_compound.name}.tif', nodata=-9999.0)
                    print(da_compound.name)

# Concatenate the data arrays
fld_da_classified = xr.concat(fld_da_classified, dim='run')
fld_da_classified['run'] = xr.IndexVariable('run', run_ids)
fld_da_classified.to_netcdf('pgw_drivers_classified_all.nc')

fld_da_compound = xr.concat(fld_da_compound, dim='run')
fld_da_compound['run'] = xr.IndexVariable('run', run_ids)
fld_da_compound.to_netcdf('pgw_compound_extent_all.nc')

# Cleanup flood area dataframe
fld_cells.index = ['no_flood', 'coastal', 'compound_coastal', 'runoff', 'compound_runoff']
fld_cells = pd.DataFrame(fld_cells)
fld_cells.to_csv('pgw_drivers_classified_all_cellCount.csv')

# Cleanup flood area dataframe
fld_cells_mask.index = ['no_flood', 'coastal', 'compound_coastal', 'runoff', 'compound_runoff', 'masked']
fld_cells_mask = pd.DataFrame(fld_cells_mask)
fld_cells_mask.to_csv('pgw_drivers_classified_all_cellCount_waterMask_rivers.csv')

fld_da_diff = xr.concat(fld_da_diff, dim='run')
fld_da_diff['run'] = xr.IndexVariable('run', run_ids)
fld_da_diff.to_netcdf('pgw_WL_maxCmpd_minus_maxIndiv_all.nc')

flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
flor_pres
floy_pres
floy_pres
floy_pres
floy_pres
floy_pres
floy_pres
floy_pres


KeyboardInterrupt: 

In [ ]:
''' Parse the drivers of the ensemble mean '''
os.chdir(r'Z:\Data-Expansion\users\lelise\projects\Carolinas_SFINCS\Chapter2_PGW\sfincs\03_OBS\analysis_2\ensemble_mean')
for type in ['mean', 'max']:
    zsmax_file = os.path.join(work_dir, 'zsmax', f'fut_ensemble_zsmax_{type}.nc')
    da_ensmean = xr.open_dataarray(zsmax_file)

    # if os.path.exists(f'pgw_drivers_classified_ensmean_{type}.nc') is False:
    fld_cells = pd.DataFrame()  # dataframe populated with total flooded area
    fld_cells_mask = pd.DataFrame()  # dataframe populated with total flooded area
    fld_da_compound = []  # populated with data arrays of the compound areas for each run
    fld_da_classified = []
    fld_da_diff = []
    run_ids = []
    for storm in ['flor', 'floy', 'matt']:
        for climate in ['fut']:
            compound_key, runoff_key, coastal_key = [f'{storm}_{climate}_compound_{type}',
                                                     f'{storm}_{climate}_runoff_{type}',
                                                     f'{storm}_{climate}_coastal_{type}']
            out = classify_zsmax_by_driver(da=da_ensmean,
                                           compound_key=compound_key, runoff_key=runoff_key,
                                           coastal_key=coastal_key, name_out=f'{storm}_{climate}_ensmean',
                                           hmin=0.05,
                                           mask=mask
                                          )
            da_classified, fld_cells_by_driver, da_compound, da_diff, fld_cells_by_driver_Mask = out

            # Append the classified data array (e.g., coastal, runoff, compound) to the larger list
            fld_da_classified.append(da_classified)
            # Append the count of flooded cells by driver to the dataframe
            fld_cells[f'{da_compound.name}'] = fld_cells_by_driver
            # Append the count of flooded cells by driver using a mask to the dataframe
            fld_cells_mask[f'{da_compound.name}'] = fld_cells_by_driver_Mask
            # Append the compound extent data array to the larger list
            fld_da_compound.append(da_compound)
            # Append the difference between the compound and the max single driver data array to the larger list
            fld_da_diff.append(da_diff)
            # Append the run ID to the larger list, used for saving as a run index
            run_ids.append(f'{da_compound.name}')
            # Write the compound extent to a tif for viewing
            da_compound.raster.to_raster(f'{da_compound.name}.tif', nodata=-9999.0)
            print(da_compound.name)

    # Concatenate the data arrays
    fld_da_compound = xr.concat(fld_da_compound, dim='run')
    fld_da_compound['run'] = xr.IndexVariable('run', run_ids)
    fld_da_compound.to_netcdf(f'pgw_compound_extent_ensmean_{type}.nc')
    
    fld_da_classified = xr.concat(fld_da_classified, dim='run')
    fld_da_classified['run'] = xr.IndexVariable('run', run_ids)
    fld_da_classified.to_netcdf(f'pgw_drivers_classified_ensmean_{type}.nc')

    fld_da_diff = xr.concat(fld_da_diff, dim='run')
    fld_da_diff['run'] = xr.IndexVariable('run', run_ids)
    fld_da_diff.to_netcdf(f'pgw_WL_maxCmpd_minus_maxIndiv_ensmean_{type}.nc')

    # Cleanup flood area dataframe
    # No Flood = 0, Coastal = 1, Compound-coastal = 2, Runoff = 3, Compound-runoff = 4
    fld_cells.index = ['no_flood', 'coastal', 'compound_coastal', 'runoff', 'compound_runoff']
    fld_cells = pd.DataFrame(fld_cells)
    fld_cells.to_csv(f'pgw_drivers_classified_ensmean_{type}_cellCount.csv')

    # Cleanup flood area dataframe
    # No Flood = 0, Coastal = 1, Compound-coastal = 2, Runoff = 3, Compound-runoff = 4, np.Nan = 5
    print(fld_cells_mask)
    fld_cells_mask.index = ['no_flood', 'coastal', 'compound_coastal', 'runoff', 'compound_runoff', 'masked']
    fld_cells_mask = pd.DataFrame(fld_cells_mask)
    fld_cells_mask.to_csv(f'pgw_drivers_classified_ensmean_{type}_cellCount_waterMask_rivers.csv')

print(os.getcwd())